# シミュレーションコード

処理の整理
- 遷移確率の算出・正解データの作成
    - 期間を定義し、parquet fileから各時間帯の遷移確率を算出する
    - データリークが起きないように期間を設定、正解データとする
- シミュレーション
    - シミュレーションパラメータの設定
        - mesh レベルの設定
        - 初期配置のPoIの設定
        - 開始時間
        - 終了時間
        - 遷移分岐の最大数
        - トップN設定



### 1：ライブラリ

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
from datetime import datetime
import gc



## 正解データ・遷移確率の算出

- 正解データ・同化データと遷移確率算出に使う期間を分けファイルをリスト化
- 遷移確率算出
    - ファイルを結合
    - 日付を無視して、時間ごとに遷移確率を算出
    - transition_prob_dfで保存　DF
- 正解・同化データの作成
    - 遷移人数から滞在人数を算出
    - gt_stay_df で保存　DF

In [2]:
# 正解データ・同化データと遷移確率算出に使う期間を分けファイルをリスト化


BASE_DIR = Path("../../data/processed/yamada_processed/trans_count")

# =========================
# 期間指定（ここだけ変える）
# =========================
TRANS_START = "20190201"  # 遷移確率（学習/同化）に使う開始日
TRANS_END   = "20190207"  # 遷移確率（学習/同化）に使う終了日

GT_START = "20190208"     # 正解（評価）に使う開始日
GT_END   = "20190208"     # 正解（評価）に使う終了日


# =========================
# state_count_YYYYMMDD.csv を集める
# =========================
all_paths = sorted(BASE_DIR.glob("state_count_*.csv"))
print("all files:", len(all_paths))

date_re = re.compile(r"state_count_(\d{8})\.csv$")


# =========================
# 日付で分ける
# =========================
trans_paths = []
gt_paths = []
others = []

trans_start = datetime.strptime(TRANS_START, "%Y%m%d")
trans_end   = datetime.strptime(TRANS_END, "%Y%m%d")

gt_start = datetime.strptime(GT_START, "%Y%m%d")
gt_end   = datetime.strptime(GT_END, "%Y%m%d")

for p in all_paths:
    m = date_re.search(p.name)
    if not m:
        others.append(p)
        continue

    d = datetime.strptime(m.group(1), "%Y%m%d")

    if trans_start <= d <= trans_end:
        trans_paths.append(p)
    elif gt_start <= d <= gt_end:
        gt_paths.append(p)
    else:
        others.append(p)


# =========================
# 結果確認
# =========================
print("\n【遷移確率（学習・同化）に使う期間】")
print("ファイル数:", len(trans_paths))
for p in trans_paths:
    print(p.name)

print("\n【正解（評価）に使う期間】")
print("ファイル数:", len(gt_paths))
for p in gt_paths:
    print(p.name)



all files: 14

【遷移確率（学習・同化）に使う期間】
ファイル数: 7
state_count_20190201.csv
state_count_20190202.csv
state_count_20190203.csv
state_count_20190204.csv
state_count_20190205.csv
state_count_20190206.csv
state_count_20190207.csv

【正解（評価）に使う期間】
ファイル数: 1
state_count_20190208.csv


In [3]:
# 遷移確率算出


df_list = []
for p in trans_paths:
    tmp = pd.read_csv(p)
    tmp["__source_file"] = p.name
    df_list.append(tmp)

trans_df = pd.concat(df_list, ignore_index=True)

print("【結合完了】行数:", len(trans_df))
print("カラム:", list(trans_df.columns))
trans_df.head()

trans_df["time_15min"] = pd.to_datetime(trans_df["time_15min"], errors="coerce")
trans_df = trans_df.dropna(subset=["time_15min"]).copy()

# 日付を無視して「時刻（HH:MM）」だけにする
trans_df["time_hm"] = trans_df["time_15min"].dt.strftime("%H:%M")

trans_df[["time_15min", "time_hm"]].head()

# 分母：ある time_hm, state_* にいる総遷移数
denom = (
    trans_df
    .groupby(["time_hm", "state_mesh", "state_poi"], as_index=False)
    .agg(total=("n_transition", "sum"))
)

# 分子：特定の next_* への遷移数（そのまま）
numer = (
    trans_df
    .groupby(["time_hm", "state_mesh", "state_poi", "next_mesh", "next_poi"], as_index=False)
    .agg(n_transition=("n_transition", "sum"))
)

# 遷移確率 p
transition_prob_df = numer.merge(
    denom, on=["time_hm", "state_mesh", "state_poi"], how="left"
)
transition_prob_df["p"] = transition_prob_df["n_transition"] / transition_prob_df["total"]

print("【遷移確率】shape:", transition_prob_df.shape)

def normalize_mesh_str(x) -> str:
    if pd.isna(x):
        return ""
    try:
        return str(int(float(x)))  # 11111111.0 -> "11111111"
    except Exception:
        return str(x).strip()

# mesh正規化
transition_prob_df["state_mesh"] = transition_prob_df["state_mesh"].apply(normalize_mesh_str)
transition_prob_df["next_mesh"]  = transition_prob_df["next_mesh"].apply(normalize_mesh_str)

# poi正規化（空白やNaN対策）
transition_prob_df["state_poi"] = transition_prob_df["state_poi"].astype(str).str.strip()
transition_prob_df["next_poi"]  = transition_prob_df["next_poi"].astype(str).str.strip()

# time正規化
transition_prob_df["time_hm"] = transition_prob_df["time_hm"].astype(str).str.zfill(5)

transition_prob_df.head()




【結合完了】行数: 200836
カラム: ['time_15min', 'state_mesh', 'state_poi', 'next_mesh', 'next_poi', 'n_transition', '__source_file']
【遷移確率】shape: (80308, 8)


,time_hm,state_mesh,state_poi,next_mesh,next_poi,n_transition,total,p
0,00:00,11111111,OoR,11111111,OoR,207,299,0.692308
1,00:00,11111111,OoR,53393464,home,1,299,0.003344
2,00:00,11111111,OoR,53393472,home,1,299,0.003344
3,00:00,11111111,OoR,53393472,move,3,299,0.010033
4,00:00,11111111,OoR,53393473,home,1,299,0.003344


In [4]:
## 正解データの準備

df_list = []
for p in gt_paths:
    tmp = pd.read_csv(p)
    tmp["__source_file"] = p.name
    df_list.append(tmp)

gt_df = pd.concat(df_list, ignore_index=True)

print("【結合完了】行数:", len(gt_df))
print("カラム:", list(gt_df.columns))
gt_df.head()

# time_15min を datetime に
gt_df["time_15min"] = pd.to_datetime(gt_df["time_15min"], errors="coerce")
gt_df = gt_df.dropna(subset=["time_15min"]).copy()

# 日付を無視して時刻(HH:MM)だけにする（必要なら）
gt_df["time_hm"] = gt_df["time_15min"].dt.strftime("%H:%M")


# ✅ 各時刻 × 各メッシュ × 各PoI の滞在人数
gt_stay_df = (
    gt_df
    .groupby(["time_hm", "state_mesh", "state_poi"], as_index=False)
    .agg(n_people=("n_transition", "sum"))
)

# =========================
# ★追加：正規化（ここが重要）
# =========================
gt_stay_df["state_mesh"] = (
    gt_stay_df["state_mesh"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)  # "53393472.0" -> "53393472"
)

gt_stay_df["state_poi"] = (
    gt_stay_df["state_poi"]
    .astype(str)
    .str.strip()
)

print("【滞在人数DF】shape:", gt_stay_df.shape)
display(gt_stay_df.head(20))


【結合完了】行数: 29630
カラム: ['time_15min', 'state_mesh', 'state_poi', 'next_mesh', 'next_poi', 'n_transition', '__source_file']
【滞在人数DF】shape: (20140, 4)


,time_hm,state_mesh,state_poi,n_people
0,00:00,11111111,OoR,57
1,00:00,11111111,home,400
2,00:00,53393463,home,3
3,00:00,53393463,建設・工事,1
4,00:00,53393464,home,14
5,00:00,53393464,販売・卸,1
6,00:00,53393471,home,5
7,00:00,53393472,home,46
8,00:00,53393472,交通・運輸・倉庫,1
9,00:00,53393473,home,68


# シミュレーションの実行

- 初期状態の作成
    - 合成人口データをロードする
        - 対象者を社会人に絞る  
    - 各個人の緯度・経度を取得する
    - 緯度・経度からメッシュIDを付与する
        - このメッシュIDを実験で使う空間単位とする
    - 初期配置として PoI = home を与える

In [5]:
# =========================
# 設定（パラメータ）
# =========================
RAW_PATH = Path("../../data/raw/population/2015_001_8_13208.csv")

MIN_AGE = 18

# 阪上加工で残る就業形態（あなたの検証から確定）
KEEP_EMPLOYMENT_TYPES = ["一般労働者", "短時間労働者", "臨時労働者"]

rename_dict = {
    "prefecture_code": "都道府県コード",
    "prefecture_name": "都道府県名",
    "city_code": "市区町村コード",
    "city_name": "市区町村名",
    "town_code": "町丁目コード",
    "town_name": "町丁目名",
    "latitude": "緯度",
    "longitude": "経度",
    "household_id": "世帯ID",
    "family_type_id": "世帯類型ID",
    "family_type": "世帯類型",
    "num_member": "世帯人数",
    "abnormal_household": "異常世帯フラグ",
    "person_id": "個人ID",
    "age": "年齢",
    "gender_id": "性別ID",
    "gender": "性別",
    "role_household_type_id": "世帯内役割ID",
    "role_household_type": "世帯内役割",
    "industry_type_id": "産業分類ID",
    "industry_type": "産業分類",
    "employment_type_id": "就業形態ID",
    "employment_type": "就業形態",
    "company_size_id": "企業規模ID",
    "company_size": "企業規模",
}

# =========================
# Step 1: raw 読み込み
# =========================
df_raw = pd.read_csv(RAW_PATH)

print("【raw】shape:", df_raw.shape)
print("【raw】age min/max:", df_raw["age"].min(), df_raw["age"].max())
print("【raw】employment_type unique:", df_raw["employment_type"].dropna().unique())

# =========================
# Step 2: 条件抽出（阪上と一致する加工）
#   - 年齢 >= 18
#   - 就業形態 ∈ {一般労働者, 短時間労働者, 臨時労働者}
# =========================
df_people = df_raw[
    (df_raw["age"] >= MIN_AGE) &
    (df_raw["employment_type"].isin(KEEP_EMPLOYMENT_TYPES))
].copy()

print("【processed】shape:", df_people.shape)
print("【processed】age min/max:", df_people["age"].min(), df_people["age"].max())
print("【processed】employment_type:", df_people["employment_type"].value_counts(dropna=False))

# =========================
# Step 3: 日本語列名に変換
# =========================
df_people = df_people.rename(columns=rename_dict)

print("人数:", len(df_people))
print("columns:", df_people.columns.tolist())

display(df_people.head(1))


【raw】shape: (214281, 25)
【raw】age min/max: 0 100
【raw】employment_type unique: ['短時間労働者' '一般労働者' '臨時労働者']
【processed】shape: (88849, 25)
【processed】age min/max: 18 100
【processed】employment_type: employment_type
一般労働者     66584
短時間労働者    21378
臨時労働者       887
Name: count, dtype: int64
人数: 88849
columns: ['都道府県コード', '都道府県名', '市区町村コード', '市区町村名', '町丁目コード', '町丁目名', '緯度', '経度', '世帯ID', '世帯類型ID', '世帯類型', '世帯人数', '異常世帯フラグ', '個人ID', '年齢', '性別ID', '性別', '世帯内役割ID', '世帯内役割', '産業分類ID', '産業分類', '就業形態ID', '就業形態', '企業規模ID', '企業規模']


,都道府県コード,都道府県名,市区町村コード,市区町村名,町丁目コード,町丁目名,緯度,経度,世帯ID,世帯類型ID,...,性別ID,性別,世帯内役割ID,世帯内役割,産業分類ID,産業分類,就業形態ID,就業形態,企業規模ID,企業規模
177,13,東京都,13208,調布市,13208003004,富士見町４丁目,35.662585,139.534513,18324823,0,...,0,男性,0,単独世帯（男性）,60.0,Ｆ 電気・ガス・熱供給・水道業,10.0,一般労働者,1000.0,1000人以上


In [6]:
# 初期配置　めっしゅID の作成

# =========================
# 0. 前提チェック
# =========================
need_cols = ["個人ID", "緯度", "経度"]
missing = [c for c in need_cols if c not in df_people.columns]
if missing:
    raise ValueError(f"df_people に必要な列がありません: {missing}")

print("df_people shape:", df_people.shape)
print("緯度欠損:", df_people["緯度"].isna().sum())
print("経度欠損:", df_people["経度"].isna().sum())

# =========================
# 1. 緯度経度を数値化
# =========================
df_people["緯度"] = pd.to_numeric(df_people["緯度"], errors="coerce")
df_people["経度"] = pd.to_numeric(df_people["経度"], errors="coerce")

# 欠損がある行は mesh を作れないので除外（必要なら残してもOK）
df_people = df_people.dropna(subset=["緯度", "経度"]).copy()

print("欠損除外後 shape:", df_people.shape)
print("緯度範囲:", df_people["緯度"].min(), "〜", df_people["緯度"].max())
print("経度範囲:", df_people["経度"].min(), "〜", df_people["経度"].max())

# =========================
# 2. 緯度経度 → 3次メッシュID（約1km）
# =========================
lat = df_people["緯度"].to_numpy()
lon = df_people["経度"].to_numpy()

# ---- 1次メッシュ（40分 × 1度）----
p = np.floor(lat * 1.5).astype(int)   # 緯度 (40分単位)
a = np.floor(lon - 100).astype(int)   # 経度 (1度単位)
mesh1 = p * 100 + a

# ---- 2次メッシュ（5分 × 7.5分）----
q = np.floor((lat * 1.5 - p) * 8).astype(int)
b = np.floor(((lon - 100) - a) * 8).astype(int)
mesh2 = mesh1 * 100 + q * 10 + b

# ---- 3次メッシュ（30秒 × 45秒）----
r = np.floor(((lat * 1.5 - p) * 8 - q) * 10).astype(int)
c = np.floor((((lon - 100) - a) * 8 - b) * 10).astype(int)
mesh3 = mesh2 * 100 + r * 10 + c

# 追加（文字列が扱いやすい。数値でもOK）
df_people["mesh_level3"] = mesh3.astype(str)

print("\n✅ mesh_level3 付与完了")
print("mesh_level3 欠損:", df_people["mesh_level3"].isna().sum())
print("mesh_level3 ユニーク数:", df_people["mesh_level3"].nunique())

display(df_people[["個人ID", "緯度", "経度", "mesh_level3"]].head(10))


df_people shape: (88849, 25)
緯度欠損: 0
経度欠損: 0
欠損除外後 shape: (88849, 25)
緯度範囲: 35.636517262871074 〜 35.683031656739075
経度範囲: 139.51738976505803 〜 139.59321140606346

✅ mesh_level3 付与完了
mesh_level3 欠損: 0
mesh_level3 ユニーク数: 36


,個人ID,緯度,経度,mesh_level3
177,40952264,35.662585,139.534513,53393492
181,40952268,35.639386,139.549963,53393463
187,40952274,35.649432,139.534088,53393472
188,40952275,35.652872,139.530387,53393482
189,40952276,35.663240,139.522585,53393491
191,40952278,35.653308,139.566247,53393485
202,40952289,35.675722,139.562320,53394414
203,40952290,35.652266,139.547767,53393483
204,40952291,35.648511,139.541675,53393473
213,40952300,35.654103,139.556280,53393484


In [7]:
TIME_HM = "06:00"
BASE_SEED_INIT_POI = 777
rng = np.random.default_rng(BASE_SEED_INIT_POI)

# 初期mesh
init_df = df_people[["個人ID", "mesh_level3"]].copy()
init_df = init_df.rename(columns={"個人ID": "agent_id", "mesh_level3": "initial_mesh"})
init_df["initial_mesh"] = init_df["initial_mesh"].astype(str)

# 06:00 正解分布：mesh×poiの人数
gt_0600 = gt_stay_df[gt_stay_df["time_hm"] == TIME_HM].copy()
gt_0600["state_mesh"] = gt_0600["state_mesh"].astype(str)
gt_0600["state_poi"]  = gt_0600["state_poi"].astype(str)

mesh_poi = (
    gt_0600.groupby(["state_mesh", "state_poi"])["n_people"]
    .sum()
    .reset_index()
)

# mesh -> (poi_labels, probs) を辞書にする
mesh_to_poi_dist = {}
for mesh, g in mesh_poi.groupby("state_mesh"):
    poi_count = g.set_index("state_poi")["n_people"]
    poi_prob = (poi_count / poi_count.sum()).to_numpy()
    poi_labels = poi_count.index.to_numpy()
    mesh_to_poi_dist[mesh] = (poi_labels, poi_prob)

# fallback（meshが正解に無い場合）
global_poi_count = gt_0600.groupby("state_poi")["n_people"].sum()
global_poi_prob  = (global_poi_count / global_poi_count.sum())
global_labels = global_poi_prob.index.to_numpy()
global_probs  = global_poi_prob.to_numpy()

# meshごとにpoi割当
init_poi = []
for mesh in init_df["initial_mesh"].values:
    if mesh in mesh_to_poi_dist:
        labels, probs = mesh_to_poi_dist[mesh]
    else:
        labels, probs = global_labels, global_probs

    init_poi.append(rng.choice(labels, p=probs))

init_df["initial_poi"] = init_poi

print("✅ init_df shape:", init_df.shape)
display(init_df.head(10))

print("✅ init_df initial_poi 分布（上位）")
display(init_df["initial_poi"].value_counts().head(20))


✅ init_df shape: (88849, 3)


,agent_id,initial_mesh,initial_poi
177,40952264,53393492,home
181,40952268,53393463,home
187,40952274,53393472,home
188,40952275,53393482,販売・卸
189,40952276,53393491,home
191,40952278,53393485,home
202,40952289,53394414,home
203,40952290,53393483,home
204,40952291,53393473,move
213,40952300,53393484,home


✅ init_df initial_poi 分布（上位）


initial_poi
home                  65631
move                   9589
その他サービス                2236
自動車・オートバイ・自転車・ドライブ     1988
建設・工事                  1572
販売・卸                   1512
官公庁・団体・福祉              1217
医療・薬・保険衛生              1170
製造・加工                   956
学校・趣味教室・図書館             851
スポーツ・趣味娯楽・レジャー          665
飲食店                     543
不動産・賃貸・展示場              376
金融・保険・証券                156
交通・運輸・倉庫                139
電気・ガス・通信・放送・新聞          132
出版・印刷                    55
旅行・観光・温泉・旅館・ホテル          48
OoR                      13
Name: count, dtype: int64

## サンプリング（検証用）

In [8]:
SAMPLE_N = init_df.shape[0]  # 全員サンプリング
SEED = 42

init_small_df = init_df.sample(n=SAMPLE_N, random_state=SEED).reset_index(drop=True)

print("✅ init_small_df shape:", init_small_df.shape)
display(init_small_df.head(10))

# 初期メッシュ分布を見る（偏り確認）
mesh_dist = (
    init_small_df
    .groupby("initial_mesh", as_index=False)
    .agg(n_agents=("agent_id", "count"))
    .sort_values("n_agents", ascending=False)
)

print("ユニークメッシュ数:", mesh_dist.shape[0])
display(mesh_dist.head(20))



✅ init_small_df shape: (88849, 3)


,agent_id,initial_mesh,initial_poi
0,41013654,53393472,home
1,41149965,53393496,home
2,41042975,53393483,move
3,40956691,53393481,home
4,41109315,53393472,home
5,41162626,53393474,home
6,41004208,53393492,home
7,41078837,53394403,home
8,41119942,53393474,move
9,41017082,53393496,move


ユニークメッシュ数: 36


,initial_mesh,n_agents
14,53393484,6781
7,53393474,6495
13,53393483,6217
15,53393485,5849
6,53393473,5658
12,53393482,5283
22,53393495,4625
16,53393486,4530
5,53393472,4461
20,53393493,4291


In [9]:
# ✅ 初期worldは必ず1個にする
w0 = init_small_df.rename(columns={"initial_mesh": "mesh"}).copy()

# poi初期値が無ければ home を入れる
if "poi" not in w0.columns:
    w0["poi"] = "home"

w0 = w0[["agent_id", "mesh", "poi"]].copy()
w0["mesh"] = w0["mesh"].astype(str)
w0["poi"]  = w0["poi"].astype(str)

worlds = [{
    "mesh": w0["mesh"].to_numpy(),
    "poi":  w0["poi"].to_numpy(),
}]

print("INIT worlds =", len(worlds))
print("agents per world =", len(worlds[0]["mesh"]))


INIT worlds = 1
agents per world = 88849


# 世界を作成する

- `transition_prob_df` を `time_hm/state_mesh/state_poi/next_mesh/next_poi` を **str に統一**し，`(time_hm, state_mesh, state_poi) → 遷移候補(next,p)` の **辞書 `trans_dict`** を作る  
- `init_small_df` から **初期状態（mesh, poi=home）** を作成し，時刻 `TIME_HM=06:00` の状態 `base_agents` を用意する  
- `world_id=0..9` について乱数seedを変え，各エージェントが **遷移確率 `p` に従って1回だけサンプリング**して `(next_mesh,next_poi)` を決定する（辞書に無ければ stay）  
- worldごとの次状態を結合して，**10個の世界の1ステップ遷移結果 `worlds_1step_df`** を得る  


In [10]:
# =========================
# 設定
# =========================
BRANCH_SIZE = 10      # 1ステップで世界を何倍にするか
TOP_K = 3             # 毎時で残す世界の数
BASE_SEED = 1000

# 同化を回す時間
START_TIME = "06:00"
END_TIME_ASSIM = "09:00"   
EVAL_TIME = "09:00"       

# 15分刻みの時刻配列を作る
time_grid = pd.date_range(
    f"2019-02-01 {START_TIME}",
    f"2019-02-01 {EVAL_TIME}",
    
    freq="15min"
)
time_hm_list = [t.strftime("%H:%M") for t in time_grid]

print("time steps:", time_hm_list[:5], "...", time_hm_list[-5:])

time steps: ['06:00', '06:15', '06:30', '06:45', '07:00'] ... ['08:00', '08:15', '08:30', '08:45', '09:00']


In [11]:
# =========================
# 正解滞在分布（時刻ごと）を dict にする
# key: time_hm -> Series(index=(mesh,poi), value=n_people)
# =========================
gt_dict = {}

for tm, g in gt_stay_df.groupby("time_hm"):
    s = (
        g.groupby(["state_mesh", "state_poi"])["n_people"]
        .sum()
    )
    gt_dict[tm] = s

print("gt_dict keys example:", list(gt_dict.keys())[:5])


gt_dict keys example: ['00:00', '00:15', '00:30', '00:45', '01:00']


In [12]:
def cosine_sim_from_series(pred_s: pd.Series, gt_s: pd.Series) -> float:
    # pred/gt を同じindex空間に揃える（unionを使わない）
    df = pd.concat(
        [
            pred_s.rename("pred"),
            gt_s.rename("gt"),
        ],
        axis=1
    ).fillna(0.0)

    x = df["pred"].to_numpy(dtype=float)
    y = df["gt"].to_numpy(dtype=float)

    nx = np.linalg.norm(x)
    ny = np.linalg.norm(y)

    if nx == 0 or ny == 0:
        return 0.0

    return float(np.dot(x, y) / (nx * ny))

In [13]:
# ✅ 遷移辞書を numpy 化して作る（高速化）
src = transition_prob_df.copy()

src["time_hm"]    = src["time_hm"].astype(str)
src["state_mesh"] = src["state_mesh"].astype(str)
src["state_poi"]  = src["state_poi"].astype(str)
src["next_mesh"]  = src["next_mesh"].astype(str)
src["next_poi"]   = src["next_poi"].astype(str)
src["p"]          = src["p"].astype(float)

trans_dict = {}
for key, g in src.groupby(["time_hm", "state_mesh", "state_poi"], sort=False):
    nm = g["next_mesh"].to_numpy(dtype=str)
    np_ = g["next_poi"].to_numpy(dtype=str)
    pr = g["p"].to_numpy(dtype=float)

    s = pr.sum()
    if s > 0:
        pr = pr / s
    else:
        # 確率が全部0なら辞書に入れない（stay扱いにする）
        continue

    trans_dict[key] = (nm, np_, pr)

print("✅ trans_dict entries:", len(trans_dict))


✅ trans_dict entries: 33654


In [ ]:
def propagate_world_agentwise(mesh_arr, poi_arr, t, rng, trans_dict):
    """
    各エージェントに対して遷移確率を当てて1回サンプルする
    （=意味を変えない）
    ただし、Pythonの無駄を消して高速化
    """
    next_mesh = mesh_arr.copy()
    next_poi  = poi_arr.copy()

    # ローカル変数束縛（Python高速化の定石）
    td_get = trans_dict.get
    choice = rng.choice

    for j in range(mesh_arr.shape[0]):
        key = (t, mesh_arr[j], poi_arr[j])
        v = td_get(key)
        if v is None:
            continue

        nm, np_, pr = v
        idx = choice(len(pr), p=pr)

        next_mesh[j] = nm[idx]
        next_poi[j]  = np_[idx]

    return next_mesh, next_poi


In [ ]:
DEBUG = False
log_rows = []

N_AGENTS = len(worlds[0]["mesh"])

def _should_print_step(t_next: str) -> bool:
    return True  # 多いなら :00 / :30 のみに

print(f"🚀 START assimilation: {START_TIME} -> {END_TIME_ASSIM}")
print(f"INIT worlds={len(worlds)}, agents/world={N_AGENTS}, BRANCH_SIZE={BRANCH_SIZE}, TOP_K={TOP_K}")

for i in range(len(time_hm_list) - 1):
    t = time_hm_list[i]
    t_next = time_hm_list[i + 1]

    if t < START_TIME:
        continue
    if t >= END_TIME_ASSIM:
        break

    if _should_print_step(t_next):
        print(f"⏱ {t} -> {t_next} | worlds(before)={len(worlds)}")

    # =========================
    # Step A: worlds を増やす（エージェント単位サンプル維持）
    # =========================
    new_worlds = []

    for w_idx, w in enumerate(worlds):
        mesh_arr = w["mesh"]
        poi_arr  = w["poi"]

        for b in range(BRANCH_SIZE):
            seed = BASE_SEED + (i * 100000) + (w_idx * 1000) + b
            rng = np.random.default_rng(seed)

            next_mesh, next_poi = propagate_world_agentwise(
                mesh_arr=mesh_arr,
                poi_arr=poi_arr,
                t=t,
                rng=rng,
                trans_dict=trans_dict,
            )

            new_worlds.append({"mesh": next_mesh, "poi": next_poi})

    worlds = new_worlds

    if _should_print_step(t_next):
        print(f"   ↳ worlds(after expand)={len(worlds)}")

    # =========================
    # Step B: 毎時(:00)で剪定
    # =========================
    if t_next.endswith(":00"):
        if t_next not in gt_dict:
            print(f"[warn] {t_next} 正解が無いので剪定スキップ")
            continue

        gt_s = gt_dict[t_next]

        scores = []
        for wid, w in enumerate(worlds):
            pred_s = (
                pd.DataFrame({"mesh": w["mesh"], "poi": w["poi"]})
                .groupby(["mesh", "poi"])
                .size()
            )
            score = cosine_sim_from_series(pred_s, gt_s)
            scores.append((wid, score))

        scores.sort(key=lambda x: x[1], reverse=True)
        top_scores = scores[:TOP_K]

        keep_ids = [wid for wid, _ in top_scores]
        worlds = [worlds[wid] for wid in keep_ids]

        print(f"\n📌 [{t_next} prune] keep TOP_K={TOP_K}")
        for rank, (wid, sc) in enumerate(top_scores, 1):
            print(f"  rank{rank}: world={wid} cosine={sc:.6f}")
        print(f"  => worlds(after prune)={len(worlds)}\n")

        log_rows.append({
            "time_hm": t_next,
            "n_worlds_after_prune": len(worlds),
            "best_cosine": top_scores[0][1],
        })

print("✅ 同化完了")
print(f"残った世界数: {len(worlds)}")


🚀 START assimilation: 06:00 -> 09:00
INIT worlds=1, agents/world=88849, BRANCH_SIZE=10, TOP_K=3
⏱ 06:00 -> 06:15 | worlds(before)=1
   ↳ worlds(after expand)=10
⏱ 06:15 -> 06:30 | worlds(before)=10
   ↳ worlds(after expand)=100
⏱ 06:30 -> 06:45 | worlds(before)=100
   ↳ worlds(after expand)=1000
⏱ 06:45 -> 07:00 | worlds(before)=1000
   ↳ worlds(after expand)=10000

📌 [07:00 prune] keep TOP_K=3
  rank1: world=3532 cosine=0.487808
  rank2: world=7092 cosine=0.487669
  rank3: world=7095 cosine=0.487086
  => worlds(after prune)=3

⏱ 07:00 -> 07:15 | worlds(before)=3
   ↳ worlds(after expand)=30
⏱ 07:15 -> 07:30 | worlds(before)=30
   ↳ worlds(after expand)=300
⏱ 07:30 -> 07:45 | worlds(before)=300
   ↳ worlds(after expand)=3000
⏱ 07:45 -> 08:00 | worlds(before)=3000


In [ ]:
# =========================
# 剪定なし
#   worlds: [{"mesh": np.ndarray, "poi": np.ndarray}, ...]
#   trans_dict: {(time_hm, mesh, poi): (next_mesh_arr, next_poi_arr, prob_arr)}
# =========================

start_eval_idx = time_hm_list.index(END_TIME_ASSIM)
end_eval_idx   = time_hm_list.index(EVAL_TIME)

N_AGENTS = len(worlds[0]["mesh"])

print(f"\n🔮 Forecast: {END_TIME_ASSIM} -> {EVAL_TIME} (no prune)")
print(f"worlds(start)={len(worlds)}")

for i in range(start_eval_idx, end_eval_idx):
    t = time_hm_list[i]
    t_next = time_hm_list[i + 1]

    print(f"⏱ {t} -> {t_next} | worlds(before)={len(worlds)}")

    new_worlds = []

    for w_idx, w in enumerate(worlds):
        mesh_arr = w["mesh"]
        poi_arr  = w["poi"]

        for b in range(BRANCH_SIZE):
            # ✅ seed衝突しないように eval用のオフセットを入れる
            seed = BASE_SEED + (999999 * 1000) + (i * 100000) + (w_idx * 1000) + b
            rng = np.random.default_rng(seed)

            next_mesh = mesh_arr.copy()
            next_poi  = poi_arr.copy()

            for j in range(N_AGENTS):
                m = mesh_arr[j]
                p = poi_arr[j]
                key = (t, m, p)

                if key in trans_dict:
                    nm, np_, pr = trans_dict[key]
                    idx = rng.choice(len(pr), p=pr)
                    next_mesh[j] = nm[idx]
                    next_poi[j]  = np_[idx]
                # else: 遷移無しなら stay

            new_worlds.append({"mesh": next_mesh, "poi": next_poi})

    worlds = new_worlds

    print(f"   ↳ worlds(after expand)={len(worlds)}")

print(f"\n✅ {EVAL_TIME} 予測世界数: {len(worlds)}")



🔮 Forecast: 09:00 -> 09:00 (no prune)
worlds(start)=3

✅ 09:00 予測世界数: 3


In [ ]:
import pandas as pd

# =========================
# 評価
# =========================
if EVAL_TIME not in gt_dict:
    raise ValueError(f"18:00 の正解滞在分布が gt_dict にありません: {EVAL_TIME}")

gt_s = gt_dict[EVAL_TIME]

scores_18 = []
for wid, w in enumerate(worlds):
    # ✅ ndarray world → DataFrame にしてから groupby
    pred_s = (
        pd.DataFrame({"mesh": w["mesh"], "poi": w["poi"]})
        .groupby(["mesh", "poi"])
        .size()
    )
    score = cosine_sim_from_series(pred_s, gt_s)
    scores_18.append((wid, score))

scores_18.sort(key=lambda x: x[1], reverse=True)

print("\n[18:00 評価]")
print("best world:", scores_18[0][0], "cosine:", scores_18[0][1])

best_world_id = scores_18[0][0]
best_world = worlds[best_world_id]

best_pred = (
    pd.DataFrame({"mesh": best_world["mesh"], "poi": best_world["poi"]})
    .groupby(["mesh", "poi"])
    .size()
    .reset_index(name="n_people_pred")
)

display(best_pred.sort_values("n_people_pred", ascending=False).head(20))



[18:00 評価]
best world: 0 cosine: 0.06903117603163085


,mesh,poi,n_people_pred
14,53393484,home,6781
7,53393474,home,6495
13,53393483,home,6217
15,53393485,home,5849
6,53393473,home,5658
12,53393482,home,5283
22,53393495,home,4625
16,53393486,home,4530
5,53393472,home,4461
20,53393493,home,4291


In [ ]:
print("\n[18:00 TOP3]")
for rank, (wid, sc) in enumerate(scores_18[:3], 1):
    print(f"  rank{rank}: world={wid} cosine={sc:.6f}")



[18:00 TOP3]
  rank1: world=0 cosine=0.069031
  rank2: world=1 cosine=0.069031
  rank3: world=2 cosine=0.069031


In [ ]:
# メモリ解放
del new_worlds
del scores
del pred_s
del best_pred

gc.collect()

NameError: name 'new_worlds' is not defined